In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.0 MB/s eta 0:00:00


In [3]:
# ===================================================================
#  Library
# ===================================================================
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.simplefilter("ignore")

import numpy as np
from scipy.optimize import minimize
import optuna
import lightgbm as lgb

In [4]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    n_seeds = 3
    n_trials = 2000
    save_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/exp/"
    data_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/"
    filename = "exp00064"

In [5]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [6]:
kun_exp00060_oof_pred= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00060_oof_pred.csv').rename(columns={'oof_pred': 'kun_pred_0'})

In [7]:
kun_oof =  kun_exp00060_oof_pred

In [8]:
kun_oof

,id,kun_pred_0
0,0,9322.824825
1,1,3891.997620
2,2,2971.556923
3,3,7959.507481
4,4,4375.457013
...,...,...
27527,27527,12719.460234
27528,27528,6836.122512
27529,27529,14140.050721
27530,27530,6563.780784


In [9]:
oof_df_exp066 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp066.csv').rename(columns={'pred': 'pred_0'})
oof_df_exp067 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp067.csv').rename(columns={'pred': 'pred_1'})
oof_df_exp068 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp068.csv').rename(columns={'pred': 'pred_2'})
oof_df_exp069 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp069.csv').rename(columns={'pred': 'pred_3'})
oof_df_exp070 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp070.csv').rename(columns={'pred': 'pred_4'})
oof_df_exp071 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp071.csv').rename(columns={'pred': 'pred_5'})
oof_df_exp077 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp077.csv').rename(columns={'pred': 'pred_6'})

In [10]:
yuji_oof = pd.concat([oof_df_exp066, oof_df_exp067['pred_1'], oof_df_exp068['pred_2'], oof_df_exp069['pred_3'], oof_df_exp070['pred_4'], oof_df_exp071['pred_5'], oof_df_exp077.sort_values(by="id")['pred_6'].reset_index(drop=True)],axis=1)

In [14]:
yuji_oof

,id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6
0,0,8854.693376,9152.341955,8780.223598,8955.701853,8946.926107,9196.765168,7579.777693
1,1,3869.563539,3864.146063,3878.933184,3892.975519,3870.646555,3885.205290,4001.311742
2,2,3020.466736,2996.637993,3037.993605,3035.039709,3021.979636,3060.125866,2899.577998
3,3,8014.619227,8068.601962,8042.945716,7945.924737,7996.754020,8007.789593,8312.877683
4,4,4522.686418,4480.841914,4437.256123,4493.480177,4469.191551,4537.125697,4522.498509
...,...,...,...,...,...,...,...,...
27527,27527,12861.324818,12358.048015,12682.396136,12568.162607,12655.494329,12588.421140,13431.394502
27528,27528,6749.685248,6667.584863,6828.261506,6849.497946,6760.195573,6918.664724,6309.261596
27529,27529,14712.069188,14581.011664,14548.452269,15346.372643,14510.932423,14326.701625,15666.845254
27530,27530,6549.350399,6893.909033,6721.431621,6817.442094,6620.145505,6748.854564,6746.399279


In [13]:
kun_exp00060 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00060.csv', header=None).rename(columns={0:'id', 1: 'kun_pred_0'})

In [15]:
kun_test =  kun_exp00060

In [16]:
kun_test

,id,kun_pred_0
0,27532,9456.567691
1,27533,5475.010419
2,27534,5480.674549
3,27535,18054.427058
4,27536,4255.354923
...,...,...
27532,55064,13098.493637
27533,55065,9205.423724
27534,55066,5584.544090
27535,55067,4811.401836


In [17]:
exp066 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp066.csv', header=None).rename(columns={0:'id', 1: 'pred_0'})
exp067 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp067.csv', header=None).rename(columns={0:'id', 1: 'pred_1'})
exp068 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp068.csv', header=None).rename(columns={0:'id', 1: 'pred_2'})
exp069 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp069.csv', header=None).rename(columns={0:'id', 1: 'pred_3'})
exp070 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp070.csv', header=None).rename(columns={0:'id', 1: 'pred_4'})
exp071 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp071.csv', header=None).rename(columns={0:'id', 1: 'pred_5'})
exp077 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp077.csv', header=None).rename(columns={0:'id', 1: 'pred_6'})

In [18]:
yuji_test = pd.concat([exp066, exp067['pred_1'], exp068['pred_2'], exp069['pred_3'], exp070['pred_4'], exp071['pred_5'], exp077['pred_6']],axis=1)

In [19]:
yuji_test

,id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6
0,27532,9673.031004,9645.690291,9773.740233,9543.305631,9574.334533,9774.453170,9017.561685
1,27533,5585.545628,5611.390293,5625.333842,5745.546668,5722.954729,5762.007883,5339.376994
2,27534,5595.254657,5581.507724,5572.766942,5592.961109,5618.968955,5606.688175,5958.715261
3,27535,18194.779659,18336.045587,18254.450168,18366.815171,18448.417408,18256.398339,18936.869485
4,27536,4416.241293,4401.738676,4296.607046,4481.441494,4280.702030,4366.343344,4107.333654
...,...,...,...,...,...,...,...,...
27532,55064,13152.559842,12960.063334,13475.780560,13120.612989,12858.658767,13356.518375,14520.403540
27533,55065,9418.373026,9643.030937,9448.358308,9692.777950,9640.506929,9442.477572,8472.134068
27534,55066,5572.741856,5591.947060,5672.228774,5506.796525,5518.031687,5606.474561,5887.029019
27535,55067,4810.341987,4775.172193,4859.511859,4862.730132,4858.379804,4816.454751,4972.316537


In [20]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')

In [21]:
df = pd.concat([kun_oof.merge(yuji_oof, on='id'), df_train['price']], axis=1)
test = kun_test.merge(yuji_test, on='id')

In [22]:
df

,id,kun_pred_0,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,price
0,0,9322.824825,8854.693376,9152.341955,8780.223598,8955.701853,8946.926107,9196.765168,7579.777693,27587
1,1,3891.997620,3869.563539,3864.146063,3878.933184,3892.975519,3870.646555,3885.205290,4001.311742,4724
2,2,2971.556923,3020.466736,2996.637993,3037.993605,3035.039709,3021.979636,3060.125866,2899.577998,10931
3,3,7959.507481,8014.619227,8068.601962,8042.945716,7945.924737,7996.754020,8007.789593,8312.877683,16553
4,4,4375.457013,4522.686418,4480.841914,4437.256123,4493.480177,4469.191551,4537.125697,4522.498509,5158
...,...,...,...,...,...,...,...,...,...,...
27527,27527,12719.460234,12861.324818,12358.048015,12682.396136,12568.162607,12655.494329,12588.421140,13431.394502,32212
27528,27528,6836.122512,6749.685248,6667.584863,6828.261506,6849.497946,6760.195573,6918.664724,6309.261596,5400
27529,27529,14140.050721,14712.069188,14581.011664,14548.452269,15346.372643,14510.932423,14326.701625,15666.845254,22227
27530,27530,6563.780784,6549.350399,6893.909033,6721.431621,6817.442094,6620.145505,6748.854564,6746.399279,3054


In [23]:
test

,id,kun_pred_0,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6
0,27532,9456.567691,9673.031004,9645.690291,9773.740233,9543.305631,9574.334533,9774.453170,9017.561685
1,27533,5475.010419,5585.545628,5611.390293,5625.333842,5745.546668,5722.954729,5762.007883,5339.376994
2,27534,5480.674549,5595.254657,5581.507724,5572.766942,5592.961109,5618.968955,5606.688175,5958.715261
3,27535,18054.427058,18194.779659,18336.045587,18254.450168,18366.815171,18448.417408,18256.398339,18936.869485
4,27536,4255.354923,4416.241293,4401.738676,4296.607046,4481.441494,4280.702030,4366.343344,4107.333654
...,...,...,...,...,...,...,...,...,...
27532,55064,13098.493637,13152.559842,12960.063334,13475.780560,13120.612989,12858.658767,13356.518375,14520.403540
27533,55065,9205.423724,9418.373026,9643.030937,9448.358308,9692.777950,9640.506929,9442.477572,8472.134068
27534,55066,5584.544090,5572.741856,5591.947060,5672.228774,5506.796525,5518.031687,5606.474561,5887.029019
27535,55067,4811.401836,4810.341987,4775.172193,4859.511859,4862.730132,4858.379804,4816.454751,4972.316537


In [24]:
def get_custom_cv(df, n_splits=5):
    # priceを小さい順に各foldに振り分ける
    df = df.sort_values(by="price", ignore_index=True)
    df["fold"] = [i for i in range(n_splits)] * (df.shape[0] // n_splits) \
                + [i for i in range(df.shape[0] % n_splits)]
    df = df.sort_values(by="id", ignore_index=True)


    for fold in range(n_splits):
        train_idx = df[df["fold"] != fold].index
        valid_idx = df[df["fold"] == fold].index
        yield train_idx, valid_idx

n_splits = 8
cv = list(get_custom_cv(df, n_splits))

In [25]:
def fit_lgbm(X, y, test_data, cv, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)
    test_list = []

    for i, (idx_train, idx_valid) in enumerate(cv):
        X_tr, y_tr = X.iloc[idx_train], y.iloc[idx_train]
        X_val, y_val = X.iloc[idx_valid], y.iloc[idx_valid]
        transformed_test_data = test_data.copy()


        drop_cols = ['id', 'price']
        X_tr.drop(drop_cols, axis=1, inplace=True)
        X_val.drop(drop_cols, axis=1, inplace=True)
        transformed_test_data.drop(['id'], axis=1, inplace=True)
        test_list.append(transformed_test_data)


        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr,
            y_tr,
            categorical_feature=categorical_cols,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(100, verbose=verbose)],
        )
        pred_i = model.predict(X_val)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = mean_absolute_percentage_error(y_val, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(y, oof_pred)
    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models, test_list




# optunaを使ったパラメータチューニング
def tuning(df, cv):
     def objective(trial):
         max_depth = trial.suggest_int("max_depth", 1, 10)
         params = {
              "max_depth": max_depth,
              "n_estimators": trial.suggest_int('n_estimators', 100, 50000),
              "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
              "num_leaves": trial.suggest_int("num_leaves", 2, 2**max_depth),
              "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1.0),
              "subsample": trial.suggest_float("subsample", 0.01, 1.0),
              "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 0, 300),
              "max_bin": trial.suggest_int("max_bin", 32, 512),
              "reg_lambda": trial.suggest_float("reg_lambda", 1e-10, 100.0),
              "task":"train",
              "objective": "mape",
              "boosting":"gbdt",
              "metric": 'mape',
              "importance_type": "gain",
              "seed" : 88,
              "verbosity": -1,


         }

         score, _, _, _ = fit_lgbm(
             X=df, y=df["price"], test_data=test, cv=cv,  categorical_cols=[], params=params, verbose=-1
         )
         return score

     study = optuna.create_study(direction="minimize")
     study.optimize(objective, n_trials=100)
     print("Number of finished trials:", len(study.trials))
     print("Best trial:", study.best_trial.params)
     return study.best_trial.params

In [26]:
lgbm_params = tuning(df, cv)

lgbm_params.update(
    {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "importance_type": "gain",
    "seed" : 88,
    "verbosity": -1,
    }
)


# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    X=df,
    y=df["price"],
    test_data=test,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

[I 2023-08-24 03:49:25,219] A new study created in memory with name: no-name-a4b9e7ea-a450-4ac7-9612-594002cfbf28


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.433144
 - fold1 - 0.4331
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[6]	valid_0's mape: 0.445654
 - fold2 - 0.4457
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.430967
 - fold3 - 0.4310
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5]	valid_0's mape: 0.445076
 - fold4 - 0.4451
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5]	valid_0's mape: 0.4333
 - fold5 - 0.4333
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.429175
 - fold6 - 0.4292
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	valid_0's mape: 0.447676
 - fold7 - 0

[I 2023-08-24 03:49:33,231] Trial 0 finished with value: 0.4376326045823424 and parameters: {'max_depth': 9, 'n_estimators': 6841, 'learning_rate': 0.614002725868359, 'num_leaves': 176, 'colsample_bytree': 0.5876333482463257, 'subsample': 0.17730147633484428, 'min_data_in_leaf': 175, 'max_bin': 104, 'reg_lambda': 36.983375700477794}. Best is trial 0 with value: 0.4376326045823424.


Early stopping, best iteration is:
[4]	valid_0's mape: 0.436066
 - fold8 - 0.4361
FINISHI: Whole Score: 0.4376
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[21]	valid_0's mape: 0.432141
 - fold1 - 0.4321
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's mape: 0.443896
 - fold2 - 0.4439
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[6]	valid_0's mape: 0.433574
 - fold3 - 0.4336
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5]	valid_0's mape: 0.446119
 - fold4 - 0.4461
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's mape: 0.432082
 - fold5 - 0.4321
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's mape: 0.428304
 - fold6 - 0.4283
Training until validati

[I 2023-08-24 03:49:39,806] Trial 1 finished with value: 0.4379117483634084 and parameters: {'max_depth': 3, 'n_estimators': 9180, 'learning_rate': 0.9958691549107603, 'num_leaves': 7, 'colsample_bytree': 0.3166775413547369, 'subsample': 0.03797640267458328, 'min_data_in_leaf': 112, 'max_bin': 450, 'reg_lambda': 96.14159824662444}. Best is trial 0 with value: 0.4376326045823424.


...

Early stopping, best iteration is:
[580]	valid_0's mape: 0.43581
 - fold8 - 0.4358
FINISHI: Whole Score: 0.4355
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[448]	valid_0's mape: 0.43063
 - fold1 - 0.4306
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[339]	valid_0's mape: 0.442701
 - fold2 - 0.4427
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[495]	valid_0's mape: 0.430243
 - fold3 - 0.4302
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[384]	valid_0's mape: 0.445051
 - fold4 - 0.4451
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1279]	valid_0's mape: 0.428208
 - fold5 - 0.4282
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[247]	valid_0's mape: 0.427073
 - fold6 - 0.4271
Training unt

[I 2023-08-24 04:29:37,437] Trial 98 finished with value: 0.43566446644359796 and parameters: {'max_depth': 1, 'n_estimators': 42971, 'learning_rate': 0.0643734374529689, 'num_leaves': 2, 'colsample_bytree': 0.10085780567592449, 'subsample': 0.7607532473092132, 'min_data_in_leaf': 53, 'max_bin': 96, 'reg_lambda': 45.74189663811582}. Best is trial 73 with value: 0.43524906439715855.


Early stopping, best iteration is:
[257]	valid_0's mape: 0.436159
 - fold8 - 0.4362
FINISHI: Whole Score: 0.4357
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[790]	valid_0's mape: 0.430779
 - fold1 - 0.4308
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[635]	valid_0's mape: 0.443171
 - fold2 - 0.4432
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1148]	valid_0's mape: 0.43026
 - fold3 - 0.4303
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[459]	valid_0's mape: 0.444546
 - fold4 - 0.4445
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1629]	valid_0's mape: 0.428793
 - fold5 - 0.4288
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[455]	valid_0's mape: 0.427559
 - fold6 - 0.4276
Training u

[I 2023-08-24 04:30:29,892] Trial 99 finished with value: 0.4358170180900311 and parameters: {'max_depth': 2, 'n_estimators': 37163, 'learning_rate': 0.029611461546133727, 'num_leaves': 2, 'colsample_bytree': 0.2043378067436059, 'subsample': 0.727033569791886, 'min_data_in_leaf': 27, 'max_bin': 207, 'reg_lambda': 43.46524168899525}. Best is trial 73 with value: 0.43524906439715855.


Early stopping, best iteration is:
[423]	valid_0's mape: 0.436015
 - fold8 - 0.4360
FINISHI: Whole Score: 0.4358
Number of finished trials: 100
Best trial: {'max_depth': 2, 'n_estimators': 31280, 'learning_rate': 0.07929151135372496, 'num_leaves': 3, 'colsample_bytree': 0.13155897804916725, 'subsample': 0.9425999426973779, 'min_data_in_leaf': 86, 'max_bin': 158, 'reg_lambda': 31.54562806163841}
 - fold1 - 0.4305
 - fold2 - 0.4425
 - fold3 - 0.4296
 - fold4 - 0.4436
 - fold5 - 0.4278
 - fold6 - 0.4275
 - fold7 - 0.4448
 - fold8 - 0.4357
FINISHI: Whole Score: 0.4352


In [27]:
lgbm_params = {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "seed" : 88,
    "importance_type": "gain",
    "random_state": 88,
    "verbose": -1,
    'max_depth': 2,
    'n_estimators': 31280,
    'learning_rate': 0.07929151135372496,
    'num_leaves': 3,
    'colsample_bytree': 0.13155897804916725,
    'subsample': 0.9425999426973779,
    'min_data_in_leaf': 86,
    'max_bin': 158,
    'reg_lambda': 31.54562806163841
}


# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    X=df,
    y=df["price"],
    test_data=test,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

 - fold1 - 0.4305
 - fold2 - 0.4425
 - fold3 - 0.4296
 - fold4 - 0.4436
 - fold5 - 0.4278
 - fold6 - 0.4275
 - fold7 - 0.4448
 - fold8 - 0.4357
FINISHI: Whole Score: 0.4352


In [ ]:
Best trial: {'max_depth': 2, 'n_estimators': 31280, 'learning_rate': 0.07929151135372496, 'num_leaves': 3, 'colsample_bytree': 0.13155897804916725, 'subsample': 0.9425999426973779, 'min_data_in_leaf': 86, 'max_bin': 158, 'reg_lambda': 31.54562806163841}
 - fold1 - 0.4305
 - fold2 - 0.4425
 - fold3 - 0.4296
 - fold4 - 0.4436
 - fold5 - 0.4278
 - fold6 - 0.4275
 - fold7 - 0.4448
 - fold8 - 0.4357
==================================================
FINISHI: Whole Score: 0.4352

In [29]:
sub_df = pd.read_csv(CFG.data_dir + "submit_sample.csv", names=["id", "price"])

In [30]:
preds = []
for transformed_test_data, model in zip(transformed_test_data_list, models_lgb):
    pred = model.predict(transformed_test_data)
    preds.append(pred)
pred = np.mean(preds, axis=0)

# submission
sub_df["price"] = pred
sub_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{CFG.filename}.csv', index=False, header=False)

In [31]:
pd.concat([df['id'], pd.DataFrame(oof_lgb, columns=['oof_pred'])], axis=1).to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{CFG.filename}_oof_pred.csv', index=False)